# Trains ia a hyperparameter Tuning for deep NN on the IRIS dataset.
0. Epoch과 batch_size
1. Hidden Layer의 Perceptron 갯수 변경
2. Hidden Layer의 갯수 조정
3. Activation Funcion 변경
4. Learning Rate 조정
5. Optimizer 변경
6. Regularization 적용(성능 L1>L2, Training속도 L2>L1)
7. 컬럼 데이터 Normalization (큰 숫자 컬럼)
8. 인코딩(숫자라 할지라도, 숫자 자체가 크고  범주형이고 종류가 몇개 안되면)
9. BatchNormalization





In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [2]:
# 5. Optimizer 변경 -1
from tensorflow.keras import optimizers 
dir(optimizers )

['Adadelta',
 'Adagrad',
 'Adam',
 'Adamax',
 'Ftrl',
 'Nadam',
 'Optimizer',
 'RMSprop',
 'SGD',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'deserialize',
 'get',
 'schedules',
 'serialize']

In [3]:
# 0. Epoch과 batch_size
batch_size = 50
num_classes = 3
epochs = 200

In [4]:
data = load_iris()
x = data.data

In [5]:
# 8. 인코딩
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
d0=[str(i) for i in data.data[:,0:1] ]
le.fit(d0)
#le.classes_ 
data.data[:,0]=le.transform(d0)

# Decoding
#le.inverse_transform([0, 0, 1, 2])

In [6]:
y = data.target
encoder = LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values

x_train, x_test, y_train, y_test = train_test_split(x, Y,
                                                    test_size=0.2,
                                                    random_state=1)

In [7]:
# 7. 컬럼 데이터 Normalization
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(x_train)
# print(fitted.data_max_ , fitted.data_min_)

x_train = min_max_scaler.transform(x_train)
x_test = min_max_scaler.transform(x_test)
print(x_test)

[[0.41176471 0.83333333 0.03389831 0.04166667]
 [0.20588235 0.20833333 0.33898305 0.41666667]
 [0.64705882 0.41666667 0.57627119 0.54166667]
 [0.29411765 0.79166667 0.05084746 0.125     ]
 [0.97058824 0.75       0.91525424 0.79166667]
 [0.55882353 0.54166667 0.62711864 0.625     ]
 [0.73529412 0.45833333 0.69491525 0.91666667]
 [0.20588235 0.75       0.15254237 0.125     ]
 [0.11764706 0.5        0.10169492 0.04166667]
 [0.73529412 0.5        0.79661017 0.91666667]
 [0.35294118 0.29166667 0.54237288 0.5       ]
 [0.29411765 0.79166667 0.11864407 0.125     ]
 [0.79411765 0.41666667 0.83050847 0.83333333]
 [0.58823529 0.5        0.59322034 0.58333333]
 [0.76470588 0.375      0.59322034 0.58333333]
 [0.02941176 0.5        0.05084746 0.04166667]
 [0.41176471 0.25       0.50847458 0.45833333]
 [0.35294118 0.41666667 0.59322034 0.58333333]
 [0.29411765 0.58333333 0.08474576 0.125     ]
 [0.47058824 0.5        0.03389831 0.04166667]
 [0.32352941 0.25       0.57627119 0.45833333]
 [0.29411765 

In [8]:
#5. Optimizer 변경 - 2 
from tensorflow.keras.optimizers import Adam
#4. Learning Rate 조정
opt=Adam(lr=0.01)


In [9]:
#6. Regularization 적용 - 1
from keras.regularizers import l2
# instantiate regularizer
reg = l2(0.001)

In [10]:
#9. BatchNormalization -2
from keras.layers import BatchNormalization

In [11]:
model = Sequential()
#1. Hidden Layer의 Perceptron 갯수 변경
model.add(Dense(10, activation='relu', input_shape=(4,)))
#2. Hidden Layer의 갯수 조정
model.add(Dense(5, activation='relu'))
#3. Activation Funcion 변경
model.add(Dense(5, activation='tanh'))
#6. Regularization 적용 - 2
model.add(Dense(5, activation='tanh', activity_regularizer=reg))
#9. BatchNormalization - 2
model.add(BatchNormalization())     
model.add(Dense(5, activation='sigmoid'))

model.add(Dense(num_classes, activation='softmax'))
model.summary()


#5. Optimizer 변경 - 3 
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer=opt
             )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                50        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 30        
_________________________________________________________________
batch_normalization (BatchNo (None, 5)                 20        
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 1

In [12]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))


Epoch 1/200
3/3 [==============================] - 0s 79ms/step - loss: 1.0741 - accuracy: 0.3583 - val_loss: 1.0648 - val_accuracy: 0.3667
Epoch 2/200
3/3 [==============================] - 0s 12ms/step - loss: 0.9701 - accuracy: 0.5917 - val_loss: 1.0675 - val_accuracy: 0.3667
Epoch 3/200
3/3 [==============================] - 0s 10ms/step - loss: 0.9272 - accuracy: 0.6750 - val_loss: 1.0628 - val_accuracy: 0.4000
Epoch 4/200
3/3 [==============================] - 0s 13ms/step - loss: 0.8892 - accuracy: 0.6917 - val_loss: 1.0542 - val_accuracy: 0.5667
Epoch 5/200
3/3 [==============================] - 0s 11ms/step - loss: 0.8555 - accuracy: 0.6917 - val_loss: 1.0372 - val_accuracy: 0.5667
Epoch 6/200
3/3 [==============================] - 0s 9ms/step - loss: 0.8246 - accuracy: 0.6917 - val_loss: 1.0199 - val_accuracy: 0.5667
Epoch 7/200
3/3 [==============================] - 0s 11ms/step - loss: 0.7875 - accuracy: 0.6917 - val_loss: 1.0024 - val_accuracy: 0.6000
Epoch 8/200
3/3 [====

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.10227538645267487
Test accuracy: 0.9333333373069763
